In [47]:
import numpy as np
import scipy
import utils
from time import time
from utils import HouseHolder, QR, SVD

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [48]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64)


# A = np.array([[1, 0, 1],
#               [2, 5**.5, 0],
#               [0, 0, 1],
#               [0, 0, 1]])
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [49]:
print(Qt @ A @ P)

[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


## Phase-II

In [50]:
m = n = 500
A = np.random.rand(m,n)

**Test Fast Multiplication for A@B, where B is upper bidiagonal matrix.**

In [51]:
B, _, _ = SVD.svd_phaseI(A)

In [52]:
numpy_mul_begin = time()
for i in range(2000):
    A@B
numpy_mul_end = time()
numpy_mul_end - numpy_mul_begin

4.257666349411011

In [53]:
fastMul_begin = time()
for i in range(2000):
    SVD.fastMult_upper_bidiagonal(A, B)
fastMul_end = time()
fastMul_end - fastMul_begin

5.428613185882568

**Test SVD**

Choose the parameter phaseII as 'A', 'B1', 'B2' to test different implementations of phase II

In [54]:
m = 100
n = 100
A = np.random.rand(m,n)

In [55]:
U, S, Vt = SVD.svd(A, phaseII='B1')
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

phaseI: 0.0137s
phaseII: 0.1049s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %


**Accuracy Test:**

In [56]:
acc = 1e-12
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

Percentage of entrices successfully recovered by SVD with accuracy: 1e-12
100.0 %


**Scipy SVD**

In [57]:
U, S, Vt  = scipy.linalg.svd(A, full_matrices=False)
# print(np.abs(U@np.diag(S)@Vt - A))
acc = 1e-13
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")
# U, S, Vt

Percentage of entrices successfully recovered by SVD with accuracy: 1e-13
100.0 %


## Test for p2

In [110]:
m = n = 100
k = 3
A = np.array(np.diag([1/2]*n, 0)+np.diag([3/8]*(n-1), 1)+np.diag([1/8]*(n-1), -1))

A = np.linalg.matrix_power(A, k)
U, S, Vt = SVD.svd(A, phaseII="A")
# U, S, Vt = scipy.linalg.svd(A)
acc = 1e-8
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(S)@Vt - A)< acc) / (n*m) * 100, "%")

phaseI: 0.0150s
phaseII: 0.1599s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
1.96 %
